In [1]:
import os, glob, csv, string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

from sklearn.metrics import r2_score, explained_variance_score

In [2]:
def make_pred_df():
    
    meco_files = glob.glob(r'../results/eye-movement-prediction/preds_normal/preds_eval-meco-English_*.tsv')
    geco_files = glob.glob(r'../results/eye-movement-prediction/preds_eval-xlm-geco-5fold_*.tsv')

    # Read in files as dataframes + add column with filenames
    meco_dfs = []
    for file in meco_files:
        df = pd.read_csv(file, sep='\t', names=['et_metric', 'pred', 'true'])
        df['pred'] = [float(pred.lstrip('[').rstrip(']')) for pred in df['pred'].tolist()]
        df['true'] = [float(true.lstrip('[').rstrip(']')) for true in df['true'].tolist()]
        df['fold'] = len(df) * [file[-5]]
        df['dataset'] = len(df) * ['meco']
        meco_dfs.append(df)
    meco_preds = pd.concat(meco_dfs, axis=0, ignore_index=True)

    geco_dfs = []
    for file in geco_files:
        df = pd.read_csv(file, sep='\t', names=['et_metric', 'pred', 'true'])
        df['pred'] = [float(pred.lstrip('[').rstrip(']')) for pred in df['pred'].tolist()]
        df['true'] = [float(true.lstrip('[').rstrip(']')) for true in df['true'].tolist()]
        df['fold'] = len(df) * [file[-5]]
        df['dataset'] = len(df) * ['geco']
        geco_dfs.append(df)
    geco_preds = pd.concat(geco_dfs, axis=0, ignore_index=True)
    
    all_preds = pd.concat([meco_preds, geco_preds], axis=0, ignore_index=True)
    
    return all_preds

In [4]:
def make_eval_df(pred_df):
    datasets = []
    et_metrics = []
    exp_var_scores = []
    r2_scores = []
    folds = []

    for (dataset, fold, et_metric), group in pred_df.groupby(['dataset', 'fold', 'et_metric']):

        # evaluate
        exp_var, r2 = evaluate(group)

        # collect information
        datasets.append(dataset)
        folds.append(fold)
        et_metrics.append(et_metric)
        exp_var_scores.append(exp_var)
        r2_scores.append(r2)

    # make df
    eval_df = pd.DataFrame()
    eval_df['dataset'] = datasets
    eval_df['fold'] = folds
    eval_df['et_metric'] = et_metrics
    eval_df['exp_var'] = exp_var_scores
    eval_df['r2'] = r2_scores
    
    return eval_df

In [5]:
eval_df = make_eval_df(make_pred_df())
stats = eval_df.groupby(['dataset', 'et_metric']).describe()
stats

exp_var                                \
                                   count      mean       std       min   
dataset et_metric                                                        
geco    scaled_first_pass_dur        5.0  0.948915  0.003007  0.945487   
        scaled_fix_count             5.0  0.931154  0.004932  0.922988   
        scaled_tot_fix_dur           5.0  0.918860  0.004370  0.913400   
        scaled_tot_regr_from_dur     5.0  0.443377  0.038699  0.400476   
meco    scaled_first_pass_dur        5.0  0.503705  0.027312  0.458806   
        scaled_fix_count             5.0  0.775441  0.016084  0.756533   
        scaled_tot_fix_dur           5.0  0.749961  0.019704  0.724785   
        scaled_tot_regr_from_dur     5.0 -0.277782  0.137205 -0.408005   

                                                                          \
                                       25%       50%       75%       max   
dataset et_metric                                                          
geco    scaled_first_pass_dur     0.946157  0.949987  0.950339  0.952607   
        scaled_fix_count          0.931025  0.931745  0.934479  0.935536   
        scaled_tot_fix_dur        0.917273  0.918511  0.919678  0.925439   
        scaled_tot_regr_from_dur  0.406485  0.457297  0.461726  0.490902   
meco    scaled_first_pass_dur     0.499063  0.512186  0.520276  0.528195   
        scaled_fix_count          0.759963  0.782967  0.785687  0.792057   
        scaled_tot_fix_dur        0.732495  0.763007  0.763018  0.766497   
        scaled_tot_regr_from_dur -0.396818 -0.318755 -0.146396 -0.118937   

                                    r2                                \
                                 count      mean       std       min   
dataset et_metric                                                      
geco    scaled_first_pass_dur      5.0  0.946292  0.005863  0.938321   
        scaled_fix_count           5.0  0.929430  0.006104  0.920984   
        scaled_tot_fix_dur         5.0  0.916878  0.005608  0.911662   
        scaled_tot_regr_from_dur   5.0  0.436575  0.046042  0.373318   
meco    scaled_first_pass_dur      5.0 -0.873324  0.267119 -1.221004   
        scaled_fix_count           5.0 -0.634844  0.345496 -1.161249   
        scaled_tot_fix_dur         5.0 -0.648711  0.240352 -0.958568   
        scaled_tot_regr_from_dur   5.0 -0.957000  0.445156 -1.706651   

                                                                          
                                       25%       50%       75%       max  
dataset et_metric                                                         
geco    scaled_first_pass_dur     0.941862  0.949236  0.950333  0.951709  
        scaled_fix_count          0.925625  0.930623  0.934385  0.935533  
        scaled_tot_fix_dur        0.912777  0.915215  0.919305  0.925429  
        scaled_tot_regr_from_dur  0.406382  0.453867  0.461099  0.488207  
meco    scaled_first_pass_dur    -1.007942 -0.896586 -0.715592 -0.525494  
        scaled_fix_count         -0.790088 -0.467700 -0.466957 -0.288225  
        scaled_tot_fix_dur       -0.819451 -0.603977 -0.497581 -0.363980  
        scaled_tot_regr_from_dur -0.981570 -0.831153 -0.677761 -0.587863